In [3]:
import mindspore
from mindspore import Tensor, nn, context
from mindspore.dataset import vision, transforms
from mindspore.dataset.vision import Inter
from mindspore.train.callback import LossMonitor

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

# 加载CIFAR-10数据集
def create_dataset(data_path, batch_size=32, repeat_size=1, training=True):
    cifar10_ds = mindspore.dataset.Cifar10Dataset(data_path)

    trans = [
        vision.Resize((32, 32), interpolation=Inter.LINEAR),
        vision.Rescale(1.0 / 255.0, 0.0),
        vision.HWC2CHW()
    ]

    if training:
        trans += [vision.RandomCrop((32, 32), (4, 4, 4, 4)), vision.RandomHorizontalFlip()]

    trans += [transforms.TypeCast(mindspore.float32)]
    cifar10_ds = cifar10_ds.map(operations=trans, input_columns="image")
    cifar10_ds = cifar10_ds.batch(batch_size, drop_remainder=True)
    cifar10_ds = cifar10_ds.repeat(repeat_size)

    return cifar10_ds

# 定义神经网络模型
class SimpleNN(nn.Cell):
    def __init__(self, num_class=10, num_channel=3):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Dense(num_channel * 32 * 32, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Dense(512, num_class)

    def construct(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 定义损失函数和优化器
net = SimpleNN(num_class=10, num_channel=3)
loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
optim = nn.Adam(net.trainable_params(), learning_rate=0.001)

# 训练模型
def train_model(train_dataset, net, loss_fn, optim, epochs=10):
    net_with_loss = nn.WithLossCell(net, loss_fn)
    train_network = nn.TrainOneStepCell(net_with_loss, optim)
    train_network.set_train()
    
    for epoch in range(epochs):
        for data in train_dataset.create_dict_iterator():
            images = data["image"]
            labels = data["label"]
            loss = train_network(images, labels)
        print(f"Epoch {epoch + 1}, Loss: {loss}")

# 加载数据集
data_path = "./cifar-10-batches-py"
train_dataset = create_dataset(data_path, batch_size=32, training=True)

# 训练模型
train_model(train_dataset, net, loss_fn, optim, epochs=10)

# 评估模型
def evaluate_model(test_dataset, net):
    net.set_train(False)
    total = 0
    correct = 0
    for data in test_dataset.create_dict_iterator():
        images = data["image"]
        labels = data["label"]
        outputs = net(images)
        predictions = outputs.argmax(axis=1)
        total += labels.shape[0]
        correct += (predictions == labels).sum().asnumpy()
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")

# 加载测试集
test_dataset = create_dataset(data_path, batch_size=32, training=False)

# 评估模型
evaluate_model(test_dataset, net)


RuntimeError: Exception thrown from dataset pipeline. Refer to 'Dataset Pipeline Error Message'. 

------------------------------------------------------------------
- Dataset Pipeline Error Message: 
------------------------------------------------------------------
[ERROR] Invalid cifar folder, cifar(.bin) files are missing under ./cifar-10-batches-py.

------------------------------------------------------------------
- C++ Call Stack: (For framework developers) 
------------------------------------------------------------------
mindspore\ccsrc\minddata\dataset\engine\datasetops\source\cifar_op.cc(223).


